In [2]:
import pandas as pd
import os, glob

In [5]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [fastparquet]


In [14]:
# testing
df_1 = pd.read_parquet("./Data/2025_09.parquet")
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458257 entries, 0 to 2458256
Data columns (total 23 columns):
 #   Column        Dtype  
---  ------        -----  
 0   currency      object 
 1   sku           float64
 2   country       object 
 3   storeid       object 
 4   sales         object 
 5   zip           object 
 6   city          object 
 7   state         float64
 8   adress        object 
 9   brand         float64
 10  lat           object 
 11  lng           object 
 12  url           object 
 13  title         object 
 14  price         object 
 15  gtin          float64
 16  varianttitle  object 
 17  breadcrumbs   object 
 18  revenue       object 
 19  priceusd      object 
 20  revenueusd    object 
 21  domain        object 
 22  dt            object 
dtypes: float64(4), object(19)
memory usage: 431.4+ MB


In [15]:
# read parquet and concat
files = [
    "./Data/2024_11.parquet", 
    "./Data/2024_12.parquet",
    "./Data/2025_01.parquet",
    "./Data/2025_02.parquet",
    "./Data/2025_03.parquet",
    "./Data/2025_04.parquet",
    "./Data/2025_05.parquet",
    "./Data/2025_06.parquet",
    "./Data/2025_07.parquet",
    "./Data/2025_08.parquet",
    "./Data/2025_09.parquet"
]

df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)

In [16]:
df.info()
# Knowing the number of rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37698167 entries, 0 to 37698166
Data columns (total 23 columns):
 #   Column        Dtype  
---  ------        -----  
 0   currency      object 
 1   sku           object 
 2   country       object 
 3   storeid       object 
 4   sales         object 
 5   zip           object 
 6   city          object 
 7   state         float64
 8   adress        object 
 9   brand         object 
 10  lat           object 
 11  lng           object 
 12  url           object 
 13  title         object 
 14  price         object 
 15  gtin          float64
 16  varianttitle  object 
 17  breadcrumbs   object 
 18  revenue       object 
 19  priceusd      object 
 20  revenueusd    object 
 21  domain        object 
 22  dt            object 
dtypes: float64(2), object(21)
memory usage: 6.5+ GB


# Phase 1

## 1. Data Cleaning

### Handle Duplicates

In [20]:
print("Before:", len(df))
df = df.drop_duplicates()
print("After:", len(df))

Before: 37698167
After: 37582071


In [ ]:
# If we want to further define which columns define uniqueness:
# example
# df = df.drop_duplicates(subset=['sku', 'storeid', 'dt'])

### Check Null Value

In [22]:
null_summary = (
    df.isna().sum()
    .to_frame("Missing_Count")
    .assign(Missing_Pct=lambda x: (x["Missing_Count"] / len(df) * 100).round(2))
    .sort_values("Missing_Count", ascending=False)
)
null_summary

,Missing_Count,Missing_Pct
state,37582071,100.00
gtin,37582071,100.00
sku,13715711,36.50
brand,12831463,34.14
breadcrumbs,32130,0.09
currency,0,0.00
country,0,0.00
city,0,0.00
zip,0,0.00
sales,0,0.00


#### Brand

In [37]:
# Decide whether fill in Unknown or lululemon
# Here chose to fill in lululemon
df['brand'] = df['brand'].fillna('lululemon')

#### sku

In [36]:
# sku
missing_dates_sku = df.loc[df['sku'].isna(), 'dt']
print("Earliest missing brand date:", missing_dates_sku.min())
print("Latest missing brand date:", missing_dates_sku.max())

Earliest missing brand date: 2025-05-21 00:00:00
Latest missing brand date: 2025-09-28 00:00:00


In [40]:
# Same sku means same title and varianttitle
# Fill in by matching other lines with same title and varianttitle
# 1.Build a mapping from (title, varianttitle) → first available SKU
sku_map = (
    df.dropna(subset=['sku'])
      .drop_duplicates(subset=['title', 'varianttitle'])
      .set_index(['title', 'varianttitle'])['sku']
      .to_dict()
)
# 2.Fill missing SKUs using that map
df['sku'] = df.apply(
    lambda x: sku_map.get((x['title'], x['varianttitle']), 'unknown')
    if pd.isna(x['sku']) else x['sku'],
    axis=1
)

In [45]:
# Verify and Check number of Unknowns
sku_summary = pd.DataFrame({
    "Null_Value": [df['sku'].isna().sum()],
    "Unknown_Value": [(df['sku'] == 'unknown').sum()],
    "Unknown_Pct": [(df['sku'] == 'unknown').mean() * 100]
}).round(2)

sku_summary

,Null_Value,Unknown_Value,Unknown_Pct
0,0,12917868,34.37


#### State

All state are null value. Try to fill in by the state value included in Address column. 

Address-first extraction (handles full names like “Ohio” and 2-letter codes like “OH”, ignores “US”)

Then fallbacks:

- ZIP → state (via pgeocode, if installed)

- City → state learned from rows that already have a state (majority per city)

- City → state using uscities.csv (only cities that are unique in the DB)

- Optional reverse geocode from lat/lng (via geopy, opt-in, with a test-time cap)

In [63]:
import re
import time
import pandas as pd
import numpy as np

# --- State dictionaries for address parsing ---
US_STATE_ABBR = {
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA",
    "ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK",
    "OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","PR","GU","VI","AS","MP"
}
STATE_NAME_TO_CODE = {
    'ALABAMA':'AL','ALASKA':'AK','ARIZONA':'AZ','ARKANSAS':'AR','CALIFORNIA':'CA','COLORADO':'CO',
    'CONNECTICUT':'CT','DELAWARE':'DE','FLORIDA':'FL','GEORGIA':'GA','HAWAII':'HI','IDAHO':'ID',
    'ILLINOIS':'IL','INDIANA':'IN','IOWA':'IA','KANSAS':'KS','KENTUCKY':'KY','LOUISIANA':'LA',
    'MAINE':'ME','MARYLAND':'MD','MASSACHUSETTS':'MA','MICHIGAN':'MI','MINNESOTA':'MN','MISSISSIPPI':'MS',
    'MISSOURI':'MO','MONTANA':'MT','NEBRASKA':'NE','NEVADA':'NV','NEW HAMPSHIRE':'NH','NEW JERSEY':'NJ',
    'NEW MEXICO':'NM','NEW YORK':'NY','NORTH CAROLINA':'NC','NORTH DAKOTA':'ND','OHIO':'OH','OKLAHOMA':'OK',
    'OREGON':'OR','PENNSYLVANIA':'PA','RHODE ISLAND':'RI','SOUTH CAROLINA':'SC','SOUTH DAKOTA':'SD',
    'TENNESSEE':'TN','TEXAS':'TX','UTAH':'UT','VERMONT':'VT','VIRGINIA':'VA','WASHINGTON':'WA',
    'WEST VIRGINIA':'WV','WISCONSIN':'WI','WYOMING':'WY','DISTRICT OF COLUMBIA':'DC',
    'PUERTO RICO':'PR','GUAM':'GU','VIRGIN ISLANDS':'VI','AMERICAN SAMOA':'AS','NORTHERN MARIANA ISLANDS':'MP'
}
FULL_STATE_PATTERN = re.compile(
    r'\b(' + '|'.join(map(re.escape, sorted(STATE_NAME_TO_CODE.keys(), key=len, reverse=True))) + r')\b',
    flags=re.IGNORECASE
)
ABBR_PATTERN = re.compile(r'\b([A-Z]{2})\b')

def _extract_state_from_address(address: str):
    """Return 2-letter state code from a free-form address. Prefers full names, then 2-letter codes; ignores 'US'."""
    if pd.isna(address):
        return None
    s = str(address)

    # 1) Try full names first (Ohio -> OH)
    full_hits = [m.group(0) for m in FULL_STATE_PATTERN.finditer(s)]
    if full_hits:
        return STATE_NAME_TO_CODE[full_hits[-1].upper()]  # last occurrence near tail

    # 2) Then try two-letter codes (e.g., ", OH,")
    tokens = [t for t in ABBR_PATTERN.findall(s.upper()) if t != "US" and t in US_STATE_ABBR]
    if tokens:
        return tokens[-1]

    return None

def _zip_to_state_factory():
    """Return a function zip->state if pgeocode is available, else a stub."""
    try:
        import pgeocode
        nomi = pgeocode.Nominatim('us')
        def _zip2state(z):
            if pd.isna(z): return np.nan
            rec = nomi.query_postal_code(str(z))
            try:
                return rec.state_code
            except Exception:
                return np.nan
        return _zip2state, True
    except Exception:
        return (lambda _z: np.nan), False

def _normalize_city(series):
    return (series.astype(str)
                  .str.replace('\xa0',' ', regex=False)
                  .str.normalize('NFKC')
                  .str.strip()
                  .str.upper())

def fill_states_address_first(
    df,
    address_col='adress',
    zip_col='zip',
    city_col='city',
    state_col='state',
    lat_col='lat',
    lng_col='lng',
    sample=None,                 # e.g., 1000 or 0.1; None = full df
    city_db_path="/mnt/data/uscities.csv",  # path to uscities.csv (or None to skip)
    use_zip_fallback=True,
    use_city_map_fallback=True,
    use_city_db_fallback=True,
    use_reverse_geocode=False,   # set True to enable lat/lng fallback
    reverse_limit=100,           # number of points to reverse-geocode (for tests)
    reverse_sleep_every=10,      # be nice to API: sleep every N lookups
    random_state=42,
    use_copy=True,
    verbose=True,
):
    """
    Address-first state filling with layered fallbacks.
    Returns: df_out, report (dict with counts & timings)
    Output 'state' is a 2-letter postal code (e.g., 'CA', 'NY').
    """
    t0 = time.time()
    dfx = df.copy() if use_copy else df

    # Optional sampling
    if sample is not None:
        if isinstance(sample, float) and 0 < sample < 1:
            dfx = dfx.sample(frac=sample, random_state=random_state)
        elif isinstance(sample, int) and sample > 0:
            dfx = dfx.sample(n=sample, random_state=random_state)
        else:
            raise ValueError("sample must be None, a positive int, or float in (0,1)")

    # Ensure 'state' column exists and is string dtype (prevents FutureWarning)
    if state_col not in dfx.columns:
        dfx[state_col] = pd.NA
    dfx[state_col] = dfx[state_col].astype('string')

    # Normalize helpers
    if zip_col in dfx.columns:
        dfx[zip_col] = dfx[zip_col].astype(str).str.extract(r'(\d{1,5})', expand=True)[0]
    if city_col in dfx.columns:
        dfx['_CITY'] = _normalize_city(dfx[city_col])
    else:
        dfx['_CITY'] = pd.Series(index=dfx.index, dtype='string')

    # --- STEP A: Address → state ---
    tA = time.time()
    beforeA = dfx[state_col].isna().sum()
    if address_col in dfx.columns:
        dfx['_state_from_addr'] = dfx[address_col].apply(_extract_state_from_address)
        maskA = dfx[state_col].isna() & dfx['_state_from_addr'].notna()
        dfx.loc[maskA, state_col] = dfx.loc[maskA, '_state_from_addr']
    afterA = dfx[state_col].isna().sum()
    filled_by_address = beforeA - afterA
    tA = time.time() - tA

    # --- STEP B: ZIP fallback ---
    tB = 0.0
    filled_by_zip = 0
    zip_enabled = False
    if use_zip_fallback and (zip_col in dfx.columns):
        tB0 = time.time()
        zip2state, zip_enabled = _zip_to_state_factory()
        maskB = dfx[state_col].isna() & dfx[zip_col].notna()
        beforeB = maskB.sum()
        dfx.loc[maskB, state_col] = dfx.loc[maskB, zip_col].apply(zip2state)
        afterB = dfx[state_col].isna().sum()
        filled_by_zip = beforeB - (afterB - (dfx[state_col].isna().sum() - afterA))  # simplified below
        # easier: just recompute delta for this step explicitly
        # but since we only care counts: (recompute safely)
        filled_by_zip = beforeB - (dfx[state_col].isna() & dfx[zip_col].notna()).sum()
        tB = time.time() - tB0

    # --- STEP C: City→state from current data (majority per city) ---
    tC = 0.0
    filled_by_city_map = 0
    if use_city_map_fallback:
        tC0 = time.time()
        known = dfx.dropna(subset=[state_col]).copy()
        if not known.empty:
            city_map = known.groupby('_CITY')[state_col].agg(lambda s: s.value_counts().idxmax()).to_dict()
            maskC = dfx[state_col].isna()
            beforeC = maskC.sum()
            dfx.loc[maskC, state_col] = dfx.loc[maskC, '_CITY'].map(city_map)
            afterC = dfx[state_col].isna().sum()
            filled_by_city_map = beforeC - afterC
        tC = time.time() - tC0

    # --- STEP D: City→state via uscities.csv (unique cities only) ---
    tD = 0.0
    filled_by_city_db = 0
    ambiguous_in_db = []
    if use_city_db_fallback and city_db_path:
        tD0 = time.time()
        try:
            usdb = pd.read_csv(city_db_path)
            ccol = 'city_ascii' if 'city_ascii' in usdb.columns else ('city' if 'city' in usdb.columns else None)
            scol = 'state_id'   if 'state_id'   in usdb.columns else ('state' if 'state' in usdb.columns else None)
            if ccol and scol:
                usdb['_CITY']  = _normalize_city(usdb[ccol])
                usdb['_STATE'] = usdb[scol].astype(str).str.strip().str.upper()
                counts = usdb.groupby('_CITY')['_STATE'].nunique()
                unique_cities = counts[counts.eq(1)].index
                city_db_map = (usdb[usdb['_CITY'].isin(unique_cities)]
                               .drop_duplicates('_CITY')
                               .set_index('_CITY')['_STATE']
                               .to_dict())
                maskD = dfx[state_col].isna()
                beforeD = maskD.sum()
                dfx.loc[maskD, state_col] = dfx.loc[maskD, '_CITY'].map(city_db_map)
                afterD = dfx[state_col].isna().sum()
                filled_by_city_db = beforeD - afterD

                present_cities = set(dfx['_CITY'].dropna().unique())
                ambiguous_in_db = sorted([c for c in present_cities
                                          if c in set(counts.index) and c not in unique_cities])[:20]
        except Exception as e:
            if verbose:
                print(f"[uscities fallback skipped] {e}")
        tD = time.time() - tD0

    # --- STEP E: Reverse geocode from lat/lng (optional) ---
    tE = 0.0
    filled_by_geo = 0
    if use_reverse_geocode and (lat_col in dfx.columns) and (lng_col in dfx.columns):
        try:
            from geopy.geocoders import Nominatim
            geolocator = Nominatim(user_agent="address_first_state_fill")

            def coord_to_state(lat, lon):
                try:
                    if pd.notna(lat) and pd.notna(lon):
                        loc = geolocator.reverse((lat, lon), language='en', timeout=10)
                        if loc and 'state' in loc.raw.get('address', {}):
                            name = loc.raw['address']['state'].upper()
                            # Map full name -> 2-letter if needed
                            return STATE_NAME_TO_CODE.get(name, name if name in US_STATE_ABBR else None)
                except Exception:
                    return None
                return None

            tE0 = time.time()
            miss_mask = dfx[state_col].isna() & dfx[lat_col].notna() & dfx[lng_col].notna()
            idxs = dfx[miss_mask].index.tolist()
            limit = min(reverse_limit if reverse_limit else len(idxs), len(idxs))
            if verbose:
                print(f"[reverse] querying {limit} coords…")
            for i, idx in enumerate(idxs[:limit]):
                st = coord_to_state(dfx.at[idx, lat_col], dfx.at[idx, lng_col])
                if st:
                    dfx.at[idx, state_col] = st
                    filled_by_geo += 1
                if reverse_sleep_every and i % reverse_sleep_every == 0:
                    time.sleep(1)  # polite
            tE = time.time() - tE0
        except Exception as e:
            if verbose:
                print(f"[reverse geocode skipped] {e}")

    # cleanup
    dfx.drop(columns=['_CITY', '_state_from_addr'], inplace=True, errors='ignore')

    report = {
        'rows_processed': int(len(dfx)),
        'timings_sec': {
            'address': round(tA, 4), 'zip': round(tB, 4),
            'city_map': round(tC, 4), 'city_db': round(tD, 4),
            'reverse': round(tE, 4), 'total': round(time.time() - t0, 4),
        },
        'filled_by_address': int(filled_by_address),
        'filled_by_zip': int(filled_by_zip),
        'filled_by_city_map': int(filled_by_city_map),
        'filled_by_city_db': int(filled_by_city_db),
        'filled_by_reverse': int(filled_by_geo),
        'remaining_missing': int(dfx[state_col].isna().sum()),
        'zip_enabled': bool(zip_enabled),
        'ambiguous_cities_in_db_present': ambiguous_in_db,
    }

    if verbose:
        print(
            f"[state-fill] rows={report['rows_processed']}  "
            f"addr={report['filled_by_address']}  "
            f"zip={report['filled_by_zip']}  "
            f"city_map={report['filled_by_city_map']}  "
            f"city_db={report['filled_by_city_db']}  "
            f"reverse={report['filled_by_reverse']}  "
            f"missing={report['remaining_missing']}  "
            f"time={report['timings_sec']['total']}s"
        )

    return dfx, report


In [67]:
# Run functions on test set
df_work, rpt = fill_states_address_first(
    df,
    sample=1000,                        # << try a small sample first
    city_db_path="./Data/uscities.csv",
    use_zip_fallback=True,
    use_city_map_fallback=True,
    use_city_db_fallback=True,
    use_reverse_geocode=False           # keep False for tests unless needed
)
rpt

[state-fill] rows=1000  addr=1000  zip=0  city_map=0  city_db=0  reverse=0  missing=0  time=10.5595s


{'rows_processed': 1000,
 'timings_sec': {'address': 0.0095,
  'zip': 0.0492,
  'city_map': 0.0711,
  'city_db': 0.372,
  'reverse': 0.0,
  'total': 10.5595},
 'filled_by_address': 1000,
 'filled_by_zip': 0,
 'filled_by_city_map': 0,
 'filled_by_city_db': 0,
 'filled_by_reverse': 0,
 'remaining_missing': 0,
 'zip_enabled': True,
 'ambiguous_cities_in_db_present': ['ALBANY',
  'ALBERTVILLE',
  'ALEXANDRIA',
  'ANCHORAGE',
  'ANNAPOLIS',
  'ARCADIA',
  'ARDMORE',
  'ARLINGTON',
  'ASHBURN',
  'ATHENS',
  'ATLANTA',
  'AUGUSTA',
  'AUSTIN',
  'BAKERSFIELD',
  'BALTIMORE',
  'BEAVERCREEK',
  'BELLEVUE',
  'BELLINGHAM',
  'BEND',
  'BERKELEY']}

In [68]:
df_work.head()

,currency,sku,country,storeid,sales,zip,city,state,adress,brand,...,title,price,gtin,varianttitle,breadcrumbs,revenue,priceusd,revenueusd,domain,dt
30352169,USD,unknown,US,10718,2,32821,Orlando,FL,"8200 Vineland Avenue, Room 1721A, Orlando, FL, US, 32821",lululemon,...,Scuba High-Rise Mini Skirt XS Heathered Core Ultra Light Grey,39.0,NaN,Scuba High-Rise Mini Skirt XS Heathered Core Ultra Light Grey,Women's Clothes > Skirts,78.0,39.0,78.0,shop.lululemon.com,2025-07-25
18908100,USD,155678627,US,10591,1,10843,myrtle beach,SC,"10843 Kings Rd, Building 6, #635, myrtle beach, SC, US, 29572",lululemon,...,Women's Alpaca Wool-Blend Cardigan Sweater,119.0,NaN,Women's Alpaca Wool-Blend Cardigan Sweater M Pink Mist,Women's Clothes > Sweaters,119.0,119.0,119.0,shop.lululemon.com,2025-04-06
37684926,USD,unknown,US,11424,1,80124,Lone Tree,CO,"8401 Park Meadows Center Drive, Suite 2150, Lone Tree, CO, US, 80124",lululemon,...,"Wunder Train High-Rise Tight 25"" 4 Heritage 365 Camo Deep Coal Multi",98.0,NaN,"Wunder Train High-Rise Tight 25"" 4 Heritage 365 Camo Deep Coal Multi",Women's Clothes > Pants > Leggings,98.0,98.0,98.0,shop.lululemon.com,2025-09-28
30642174,USD,unknown,US,10120,2,92660,Newport Beach,CA,"Unit 927 - 401 Newport Center Dr, Newport Beach, CA 92660, Newport Beach, CA, US, 92660",lululemon,...,"lululemon Align No Line™ High-Rise Pant 28"" 0 Black",108.0,NaN,"lululemon Align No Line™ High-Rise Pant 28"" 0 Black",Women's Clothes > Pants > Leggings,216.0,108.0,216.0,shop.lululemon.com,2025-07-27
18222422,USD,117376411,US,10229,2,11746,Huntington Station,NY,"160 Walt Whitman Road, Space 1050B, Huntington Station, NY, US, 11746",lululemon,...,"lululemon Align™ High-Rise Pant 25""",98.0,NaN,"lululemon Align™ High-Rise Pant 25"" 2 Black",Women's Clothes > Pants > Leggings,196.0,98.0,196.0,shop.lululemon.com,2025-04-01


In [69]:
# Run functions on full dataset
df_full, rpt_full = fill_states_address_first(
    df,
    sample=None,                        # full data
    city_db_path="./Data/uscities.csv",
    use_zip_fallback=True,
    use_city_map_fallback=True,
    use_city_db_fallback=True,
    use_reverse_geocode=False           # set True only if necessary (slower)
)
rpt_full

[state-fill] rows=37582071  addr=37582071  zip=0  city_map=0  city_db=0  reverse=0  missing=0  time=397.7027s


{'rows_processed': 37582071,
 'timings_sec': {'address': 306.9808,
  'zip': 6.7852,
  'city_map': 29.8892,
  'city_db': 5.3855,
  'reverse': 0.0,
  'total': 397.7027},
 'filled_by_address': 37582071,
 'filled_by_zip': 0,
 'filled_by_city_map': 0,
 'filled_by_city_db': 0,
 'filled_by_reverse': 0,
 'remaining_missing': 0,
 'zip_enabled': True,
 'ambiguous_cities_in_db_present': ['ALBANY',
  'ALBERTVILLE',
  'ALEXANDRIA',
  'ANCHORAGE',
  'ANNAPOLIS',
  'ARCADIA',
  'ARDMORE',
  'ARLINGTON',
  'ASHBURN',
  'ATHENS',
  'ATLANTA',
  'AUBURN',
  'AUGUSTA',
  'AURORA',
  'AUSTIN',
  'AVALON',
  'BAKERSFIELD',
  'BALTIMORE',
  'BEAVERCREEK',
  'BELLEVUE']}

In [71]:
null_summary_final = (
    df_full.isna().sum()
    .to_frame("Missing_Count")
    .assign(Missing_Pct=lambda x: (x["Missing_Count"] / len(df) * 100).round(2))
    .sort_values("Missing_Count", ascending=False)
)
null_summary_final

,Missing_Count,Missing_Pct
gtin,37582071,100.00
breadcrumbs,32130,0.09
currency,0,0.00
storeid,0,0.00
sales,0,0.00
sku,0,0.00
country,0,0.00
city,0,0.00
zip,0,0.00
state,0,0.00


Only gtin and breadcrumbs left null value

### Convert data types

In [74]:
numeric_cols = ['price', 'sales', 'revenue', 'priceusd', 'revenueusd']
for col in numeric_cols:
    df_full[col] = pd.to_numeric(df_full[col], errors='coerce')

df_full['dt'] = pd.to_datetime(df_full['dt'], errors='coerce')

In [77]:
for col in ['currency', 'country', 'storeid', 'city', 'brand', 'domain']:
    df_full[col] = df_full[col].astype('category')

In [78]:
df_full['lat'] = pd.to_numeric(df_full['lat'], errors='coerce')
df_full['lng'] = pd.to_numeric(df_full['lng'], errors='coerce')

In [79]:
df_full.rename(columns={'adress': 'address'}, inplace=True)

In [80]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37582071 entries, 0 to 37698166
Data columns (total 23 columns):
 #   Column        Dtype         
---  ------        -----         
 0   currency      category      
 1   sku           object        
 2   country       category      
 3   storeid       category      
 4   sales         int64         
 5   zip           object        
 6   city          category      
 7   state         string        
 8   address       object        
 9   brand         category      
 10  lat           float64       
 11  lng           float64       
 12  url           object        
 13  title         object        
 14  price         float64       
 15  gtin          float64       
 16  varianttitle  object        
 17  breadcrumbs   object        
 18  revenue       float64       
 19  priceusd      float64       
 20  revenueusd    float64       
 21  domain        category      
 22  dt            datetime64[ns]
dtypes: category(6), datetime64[ns](1), 

### Check for outliers

In [76]:
df_full[numeric_cols].describe()

,price,sales,revenue,priceusd,revenueusd
count,3.758207e+07,3.758207e+07,3.758207e+07,3.758207e+07,3.758207e+07
mean,7.892134e+01,1.240814e+00,9.531230e+01,7.892134e+01,9.531230e+01
std,3.669389e+01,1.114458e+00,8.175588e+01,3.669389e+01,8.175588e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.400000e+01,1.000000e+00,5.800000e+01,5.400000e+01,5.800000e+01
50%,6.800000e+01,1.000000e+00,7.800000e+01,6.800000e+01,7.800000e+01
75%,9.800000e+01,1.000000e+00,1.180000e+02,9.800000e+01,1.180000e+02
max,6.980000e+02,5.980000e+02,2.272400e+04,6.980000e+02,2.272400e+04


#### Sales (check top sales product)

In [85]:
df_sales = df_full.copy()
df_sales.sort_values(by='sales', ascending=False)[['breadcrumbs','title','price', 'sales', 'revenue','city']].head(5)

,breadcrumbs,title,price,sales,revenue,city
34182752,Accessories > Bags,Everywhere Belt Bag 1L ONE SIZE Black/Black,38.0,598,22724.0,San Diego
2211299,Accessories > Bags,Never Lost Keychain *Wordmark,14.0,362,5068.0,Tulalip
33895543,Accessories > Bags,Everywhere Belt Bag 1L ONE SIZE Black/Black,38.0,323,12274.0,Bloomington
16657884,Accessories > Bags,Everywhere Belt Bag 1L,38.0,314,11932.0,San Diego
33807402,Accessories > Bags,Everywhere Belt Bag 1L ONE SIZE Black/Black,38.0,309,11742.0,Bloomington


#### Price (check max price)

In [88]:
df_price = df_full.copy()
df_price[df_price['price'] > 200].sort_values(by='price', ascending=False)[['breadcrumbs','title','price','sales','revenue','city']].head(5)

,breadcrumbs,title,price,sales,revenue,city
37248563,Women's Clothes > Coats & Jackets,Snow Warrior 600-Down-Fill Long Parka 0 Black,698.0,1,698.0,Bellevue
37410919,Women's Clothes > Coats & Jackets,Snow Warrior 600-Down-Fill Long Parka 6 Black,698.0,1,698.0,Madison
34773894,Women's Clothes > Coats & Jackets,Snow Warrior 600-Down-Fill Long Parka 0 Black,698.0,1,698.0,Mission Viejo
29927826,Women's Clothes > Coats & Jackets,Snow Warrior 600-Down-Fill Long Parka 2 Black,698.0,1,698.0,Woodbury
4835652,Men's Clothes > Coats & Jackets,Cold City 600-Down-Fill Parka,598.0,1,598.0,Bethesda


#### Zeros

In [89]:
(df_full[numeric_cols] == 0).sum()

price         256
sales           0
revenue       256
priceusd      256
revenueusd    256
dtype: int64

In [94]:
zero_rows = df_full[
    (df_full['price'] == 0) &
    (df_full['revenue'] == 0) &
    (df_full['priceusd'] == 0) &
    (df_full['revenueusd'] == 0)
]

zero_rows['title'].value_counts().head(20)

title
Surge Jogger *Regular                                199
Fast and Free High-Rise Tight 25" 3 Pockets *Glow     56
Stretch Cotton Long-Sleeve Polo Shirt                  1
Name: count, dtype: int64

### Reorder and sort for clarity

#### Reorder

In [96]:
# define new logical column order
new_order = [
    # time + identifiers
    'dt', 'sku', 'storeid', 'country', 'state', 'city', 'zip',
    # store & location
    'address', 'lat', 'lng', 'domain',
    # product info
    'brand', 'breadcrumbs', 'title', 'varianttitle', 'gtin', 'currency',
    # sales & price
    'sales', 'price', 'priceusd', 'revenue', 'revenueusd',
    # misc / tracking
    'url'
]

# apply reorder (keeping any unexpected columns at the end just in case)
cols_in_df = [c for c in new_order if c in df_full.columns] + [c for c in df_full.columns if c not in new_order]
df_full = df_full[cols_in_df]

# verify
df_full.head()

,dt,sku,storeid,country,state,city,zip,address,lat,lng,...,title,varianttitle,gtin,currency,sales,price,priceusd,revenue,revenueusd,url
0,2024-11-11,153888413,11003,US,MN,Minneapolis,55401,"134 3rd Ave North, Minneapolis, MN, US, 55401",44.985615,-93.271270,...,Soft Jersey Short-Sleeve Shirt,Soft Jersey Short-Sleeve Shirt L Heathered Dramatic Magenta/Heathered Nightfall,NaN,USD,1,29.0,29.0,29.0,29.0,https://shop.lululemon.com/p/mens-t-shirts/Soft-Jersey-Short-Sleeve-Shirt-MD/_/prod11570464
1,2024-11-11,154975618,10670,US,NY,New York,10036,"592 5th Avenue, New York, NY, US, 10036",40.757310,-73.978578,...,Down for It All Jacket,Down for It All Jacket 8 True Navy,NaN,USD,1,228.0,228.0,228.0,228.0,https://shop.lululemon.com/p/jackets-and-hoodies-jackets/Down-For-It-All-Jacket/_/prod9201505
2,2024-11-11,154975618,10312,US,IL,Oak Brook,60523,"82 Oakbrook Center, Oak Brook, IL, US, 60523",41.849449,-87.953366,...,Down for It All Jacket,Down for It All Jacket 8 True Navy,NaN,USD,1,228.0,228.0,228.0,228.0,https://shop.lululemon.com/p/jackets-and-hoodies-jackets/Down-For-It-All-Jacket/_/prod9201505
3,2024-11-11,154975394,19027,US,PA,Pittsburgh,15241,"301 South Hills Village, suite 1400, Pittsburgh, PA, US, 15241",40.343523,-80.055343,...,Another Mile Jacket,Another Mile Jacket 8 Twilight Rose,NaN,USD,1,228.0,228.0,228.0,228.0,https://shop.lululemon.com/p/jackets-and-hoodies-jackets/Another-Mile-Jacket/_/prod11680480
4,2024-11-11,154975394,10313,US,IL,Northbrook,60062,"2106 Northbrook Court Ct, Unit 2176, Northbrook, IL, US, 60062",42.150018,-87.816144,...,Another Mile Jacket,Another Mile Jacket 8 Twilight Rose,NaN,USD,1,228.0,228.0,228.0,228.0,https://shop.lululemon.com/p/jackets-and-hoodies-jackets/Another-Mile-Jacket/_/prod11680480


#### Sort by Time

In [97]:
# Sort by dt
df_full = df_full.sort_values(by='dt', ascending=True).reset_index(drop=True)

In [119]:
df_full['breadcrumbs'].head(10)

0    Men's Clothes > Shirts > T-Shirts
1                          Accessories
2                          Accessories
3                          Accessories
4                          Accessories
5                          Accessories
6                   Accessories > Bags
7                   Accessories > Bags
8                   Accessories > Bags
9                   Accessories > Bags
Name: breadcrumbs, dtype: object

In [ ]:
# If we want a subset later
# mask = (df_full['dt'] >= '2025-03-01') & (df_full['dt'] <= '2025-03-31')
# march_df = df_full.loc[mask]

### Final Cleaned Dataset

In [128]:
df_full

,dt,sku,storeid,country,state,city,zip,address,lat,lng,...,title,varianttitle,gtin,currency,sales,price,priceusd,revenue,revenueusd,url
0,2024-11-11,153888413,11003,US,MN,Minneapolis,55401,"134 3rd Ave North, Minneapolis, MN, US, 55401",44.985615,-93.271270,...,Soft Jersey Short-Sleeve Shirt,Soft Jersey Short-Sleeve Shirt L Heathered Dramatic Magenta/Heathered Nightfall,NaN,USD,1,29.0,29.0,29.0,29.0,https://shop.lululemon.com/p/mens-t-shirts/Soft-Jersey-Short-Sleeve-Shirt-MD/_/prod11570464
1,2024-11-11,143482868,10410,US,MA,Boston,02116,"208 Newbury Street, Unit 206, Boston, MA, US, 02116",42.349860,-71.080400,...,Women's Power Stride Crew Socks *3 Pack,Women's Power Stride Crew Socks *3 Pack M White,NaN,USD,1,228.0,48.0,228.0,48.0,https://shop.lululemon.com/p/accessories/Power-Stride-Crew-Sock-3-Pack/_/prod11020298
2,2024-11-11,143482868,10845,US,NY,Staten Island,10314,"2655 Richmond Ave, unit 2105, Staten Island, NY, US, 10314",40.581841,-74.166866,...,Women's Power Stride Crew Socks *3 Pack,Women's Power Stride Crew Socks *3 Pack M White,NaN,USD,1,228.0,48.0,228.0,48.0,https://shop.lululemon.com/p/accessories/Power-Stride-Crew-Sock-3-Pack/_/prod11020298
3,2024-11-11,143482868,11925,US,CO,Fort Collins,80525,"3260 South College Avenue, Suite 110, Fort Collins, CO, US, 80525",40.542738,-105.076520,...,Women's Power Stride Crew Socks *3 Pack,Women's Power Stride Crew Socks *3 Pack M White,NaN,USD,1,228.0,48.0,228.0,48.0,https://shop.lululemon.com/p/accessories/Power-Stride-Crew-Sock-3-Pack/_/prod11020298
4,2024-11-11,143482868,10919,US,TX,Houston,77056,"5015 Westheimer (Galleria I), Suite A-3473, Houston, TX, US, 77056",29.738967,-95.463121,...,Women's Power Stride Crew Socks *3 Pack,Women's Power Stride Crew Socks *3 Pack M White,NaN,USD,1,228.0,48.0,228.0,48.0,https://shop.lululemon.com/p/accessories/Power-Stride-Crew-Sock-3-Pack/_/prod11020298


## 2. Regional Store Performance Segmentation

### Aggregate for Store-level Metrics

In [131]:
# Group by Store ID
def summarize_stores(df):
    # make sure columns exist
    required_cols = ['storeid', 'sku', 'varianttitle', 'sales', 'revenueusd', 'priceusd', 'state', 'city']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing column: {col}")
    
    # helper to count skus (including unknown)
    def count_skus_inclusive(x):
        known_skus = x.loc[x['sku'] != 'unknown', 'sku'].nunique()
        unknown_variants = x.loc[x['sku'] == 'unknown', 'varianttitle'].nunique()
        return known_skus + unknown_variants
    
    store_summary = (
        df.groupby('storeid', as_index=False)
        .apply(lambda x: pd.Series({
            'total_sales': x['sales'].sum(),
            'total_revenue': x['revenueusd'].sum(),
            'avg_price': x['priceusd'].mean(),
            'sku_count': count_skus_inclusive(x),
            'transaction_count': len(x),
            'state': x['state'].mode().iat[0] if not x['state'].mode().empty else np.nan,
            'city': x['city'].mode().iat[0] if not x['city'].mode().empty else np.nan
        }))
    )

    # derived metrics
    store_summary['rev_per_sku'] = store_summary['total_revenue'] / store_summary['sku_count'].replace(0, np.nan)
    store_summary['rev_per_transaction'] = store_summary['total_revenue'] / store_summary['transaction_count'].replace(0, np.nan)

    return store_summary

In [132]:
store_summary = summarize_stores(df_full)
store_summary.head()

/tmp/ipykernel_202919/1742112342.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('storeid', as_index=False)
/tmp/ipykernel_202919/1742112342.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,storeid,total_sales,total_revenue,avg_price,sku_count,transaction_count,state,city,rev_per_sku,rev_per_transaction
0,10110,97917,7702210.0,79.696158,29476,84952,CA,Santa Monica,261.304451,90.665435
1,10116,57925,4647923.0,80.943484,20932,51543,CA,Los Angeles,222.048681,90.175640
2,10120,200008,16056342.0,81.671892,38086,162209,CA,Newport Beach,421.581211,98.985519
3,10122,161146,12900681.0,81.788675,37947,136761,CA,Costa Mesa,339.965768,94.330116
4,10126,202395,15687078.0,80.058575,37429,156926,CA,San Diego,419.115606,99.964811


Store Summary Col Description
- storeid: Unique identifier for each store; used to group and analyze store-level performance.
- total_sales: Total quantity of items sold by the store.
- total_revenue: Total revenue generated by the store across all transactions.
- avg_price: Average selling price of products sold at the store.
- sku_count: Number of distinct products (unique SKUs) sold at the store.
- transaction_count: Total number of transactions recorded for the store.
- state: U.S. state where the store is located.
- city: City where the store is located.
- rev_per_sku: Average revenue generated per unique product (SKU) sold by the store.
- rev_per_transaction: Average revenue generated per transaction at the store.

In [133]:
# Export and move to Tableau
store_summary.to_csv("./Data/store_summary.csv", index=False)